In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import transforms
import zookeeper as zk
import os
from tqdm import tqdm
from PIL import Image
import numpy as np

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('spada e s' + str(device))

mappy = False ### if the Net isn't mapped
rgb = False

spada e scuda


In [3]:
class GalaxyJungle(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, mappy=False, is_rgb=False):
        self.rgb = is_rgb
        self.mappy = mappy
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        
    
    def __len__(self): return (self.img_labels).shape[0]

    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[idx, 0])) + '.jpg'
        image = Image.open(img_path)
        if not self.rgb: image = image.convert('L')
        if self.transform: image = self.transform(image)
        
        label = self.img_labels.iloc[idx, 1:]        
        label = torch.tensor(label.values, dtype=torch.float32)
        if self.mappy: label = zk.mappy(label)

        gal_id = self.img_labels.iloc[idx, 0]

        return image, label, gal_id

In [4]:
class GalaxyNet(nn.Module):
    def __init__(self, activation, initialization=False, mappy=False, is_rgb=False):
        super().__init__()
        
        self.mappy = mappy
        rgb = 3 if is_rgb else 1
        input_size = 128
        num_labels = 37
        self.loss_dict = {'batch' : [], 'epoch' : [], 'vbatch' : [], 'vepoch' : []}
        self.activation = activation
        
        self.convs = nn.Sequential(
            nn.Conv2d(rgb, 16, 3, padding='same', bias=False),
            self.activation(),
            nn.BatchNorm2d(16),

            nn.Conv2d(16, 16, 3, padding='same', bias=False),
            self.activation(),
            nn.BatchNorm2d(16),
            
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding='same', bias=False),
            self.activation(),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 32, 3, padding='same', bias=False),
            self.activation(),
            nn.BatchNorm2d(32),

            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, bias=False),
            self.activation(),
            nn.BatchNorm2d(64),

            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, bias=False),
            self.activation(),
            nn.BatchNorm2d(128),

            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, bias=False),
            self.activation(),
            nn.BatchNorm2d(256),

            nn.MaxPool2d(2)
            )

        for layer in self.convs:
            if layer.__class__.__name__ == 'Conv2d': input_size = zk.convool_size(input_size, 3, 1, 'same' if layer.padding == 'same' else 0)
            elif layer.__class__.__name__ == 'MaxPool2d': input_size = zk.convool_size(input_size, 2, 2)

        if input_size < 2: raise ValueError('You shrank too much dude.')
        print(f'Convs output size: {input_size}')

        input_linear = 256 * input_size * input_size
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_linear, 100),
            self.activation(),
            nn.Linear(100, num_labels)
            )
        
        if initialization: self.init_weights()


    def forward(self, x):
        x = self.convs(x)
        x = self.fc(x)
        if self.mappy: x = zk.mappy2D(x)
        return x


    def init_weights(self):
        if self.activation == nn.ReLU:
            nonlin = 'relu'
            a = 0
        elif self.activation == nn.LeakyReLU:
            nonlin = 'leaky_relu'
            a = .01
        
        for layer in self.convs: 
            if layer.__class__.__name__ == 'Conv2d': nn.init.kaiming_normal_(layer.weight, a=a, nonlinearity=nonlin)        

        for i in (1, -1): nn.init.constant_(self.fc[i].bias, 0)
 
        nn.init.kaiming_normal_(self.fc[1].weight, a=a, nonlinearity=nonlin)
        nn.init.xavier_uniform_(self.fc[-1].weight)      
        

    def log_the_loss(self, item,epoch=False):
        train = self.__getstate__()['training']
        if epoch and train:
            self.loss_dict['epoch'].append(item)
        elif not epoch and train:
            self.loss_dict['batch'].append(item)
        elif not train and epoch:
            self.loss_dict['vepoch'].append(item)
        elif not train and not epoch:
            self.loss_dict['vbatch'].append(item)
        return item


In [17]:
model = GalaxyNet(nn.LeakyReLU, initialization=False, mappy=False, is_rgb=rgb).to(device)
loss_function = nn.MSELoss()

## NOTE: the loss_dict will be empty, remember which is the last loss_{}.pickle file of previous trainings to append the values all together
loader = torch.load('/home/teobaldo/Uni/LCP-B/proj/PADELSCORE/model.pt', weights_only=True)
model.load_state_dict(loader['model_state_dict'])

Convs output size: 2


KeyError: 'model_state_dict'

In [18]:
model = GalaxyNet(nn.LeakyReLU, initialization=False, mappy=False, is_rgb=rgb).to(device)
loss_function = nn.MSELoss()

## NOTE: the loss_dict will be empty, remember which is the last loss_{}.pickle file of previous trainings to append the values all together
loader = torch.load('/home/teobaldo/Uni/LCP-B/proj/PADELSCORE/model.pt', weights_only=True)
model.load_state_dict(loader)

Convs output size: 2


<All keys matched successfully>

In [27]:
transform = transforms.Compose([
    transforms.ToTensor(), # Riscala le immagini tra 0 e 1
    transforms.CenterCrop(324),
    transforms.Resize(128),
])

DS = GalaxyJungle('../data/test/test_solutions_rev1.csv', '../data/test/', transform, mappy=True, is_rgb=rgb)
test_loader = DataLoader(DS, batch_size=512, shuffle=False, num_workers=os.cpu_count())   

tens=torch.empty((0,18),dtype=torch.float32)

model.eval()
loss = 0
with torch.no_grad():
        for i, vdata in tqdm(enumerate(test_loader)):
            inputs,labels, GalaxyID = vdata
            inputs,labels= inputs.to(device), labels.to(device)
            outputs = model(inputs)
            outputs = nn.ReLU()(outputs)
            outputs = zk.mappy2D(outputs)
            loss += np.sqrt(nn.MSELoss()(outputs, labels).item())
            GalaxyID = GalaxyID.unsqueeze(1)
            batch = torch.cat((GalaxyID,outputs.cpu()), dim=-1)
            tens = torch.cat((tens,batch),dim=0)
print('Test loss', loss/len(test_loader))


13it [00:20,  1.58s/it]

Test loss 0.06107108467544587


In [28]:
training_sample = zk.mappy_df(pd.read_csv('../data/validation/validation_solutions_rev1.csv'))
training_sample['train_label'] = training_sample.iloc[:,1:].idxmax(axis=1)
training_sample['train_value'] = training_sample.iloc[:,1:-1].max(axis=1)
display(training_sample[training_sample['train_value'] > .5].groupby('train_label').size())

/home/teobaldo/Uni/LCP-B/proj/GalaxyClassifier/zookeeper.py:187: RuntimeWarning: invalid value encountered in divide
  class4_2 / class4_2.sum(axis=1, keepdims=True),
/home/teobaldo/Uni/LCP-B/proj/GalaxyClassifier/zookeeper.py:190: RuntimeWarning: invalid value encountered in divide
  class5_4 / class5_4.sum(axis=1, keepdims=True),


train_label
A              7
E0          1563
E3          1454
E6            99
S0a_eon      315
SAb           40
SAc          219
SAd            1
SB0a_eon       5
SBb           25
SBc           19
Scd_eon      133
SoA          561
SoB           74
dtype: int64

In [29]:
training_sample = zk.mappy_df(pd.read_csv('../data/training/training_solutions_rev1.csv'))

training_sample['train_label'] = training_sample.iloc[:,1:].idxmax(axis=1)
training_sample['train_value'] = training_sample.iloc[:,1:-1].max(axis=1)
display(training_sample[training_sample['train_value'] > .5].groupby('train_label').size())

/home/teobaldo/Uni/LCP-B/proj/GalaxyClassifier/zookeeper.py:187: RuntimeWarning: invalid value encountered in divide
  class4_2 / class4_2.sum(axis=1, keepdims=True),
/home/teobaldo/Uni/LCP-B/proj/GalaxyClassifier/zookeeper.py:190: RuntimeWarning: invalid value encountered in divide
  class5_4 / class5_4.sum(axis=1, keepdims=True),


train_label
A             32
E0          5997
E3          5854
E6           424
S0a_eon     1383
SAa            1
SAb          207
SAc          781
SAd           10
SB0a_eon      23
SBb          115
SBc          130
SBd            3
Scd_eon      590
SoA         2148
SoB          290
dtype: int64

In [30]:
train_results = pd.DataFrame(tens, columns=['GalaxyID','E0','E3','E6','S0a_eon','SB0a_eon','Scd_eon','SoB','SoA','SAa','SAb','SAc','SAd','SBa','SBb','SBc','SBd','A'])
train_results[train_results < 0] = 0
train_results['GalaxyID'] = train_results['GalaxyID'].astype(int)
train_results.sort_values('GalaxyID', inplace=True, ignore_index=True)

true_results = zk.mappy_df(pd.read_csv('../data/test/test_solutions_rev1.csv'))
true_results.sort_values('GalaxyID', inplace=True, ignore_index=True)


train_results['train_label'] = train_results.iloc[:,1:].idxmax(axis=1)
train_results['train_value'] = train_results.iloc[:,1:-1].max(axis=1)

true_results['true_label'] = true_results.iloc[:,1:].idxmax(axis=1)
true_results['true_value'] = true_results.iloc[:,1:-1].max(axis=1)

/home/teobaldo/Uni/LCP-B/proj/GalaxyClassifier/zookeeper.py:187: RuntimeWarning: invalid value encountered in divide
  class4_2 / class4_2.sum(axis=1, keepdims=True),
/home/teobaldo/Uni/LCP-B/proj/GalaxyClassifier/zookeeper.py:190: RuntimeWarning: invalid value encountered in divide
  class5_4 / class5_4.sum(axis=1, keepdims=True),


In [31]:
true_new = true_results
# true_new = true_results[true_results['true_value'] > 0.6]
#train_new=train_results[train_results['train_value']>0.4]
confronto=pd.merge(train_results[['GalaxyID','train_label']], true_new[['GalaxyID','true_label']], on='GalaxyID')
total_count=confronto.groupby('true_label').size().reset_index(name='total_count')
mislabels = confronto[confronto['train_label'] != confronto['true_label']]

count=mislabels.groupby('true_label').size().reset_index(name='num_misclassified').sort_values(by='num_misclassified', ascending=False)
final_count= pd.merge(total_count, count, on='true_label', how='left').sort_values(by='num_misclassified', ascending=False)
final_count['score'] = (np.round((1 - final_count['num_misclassified'] / final_count['total_count']),2)*100)
final_count.sort_values(by='score', ascending=False, inplace=True)
display(final_count)



,true_label,total_count,num_misclassified,score
1,E0,1273,105,92.0
2,E3,1537,265,83.0
4,S0a_eon,408,109,73.0
7,SAc,531,148,72.0
14,Scd_eon,169,61,64.0
3,E6,290,136,53.0
12,SBc,116,58,50.0
16,SoB,197,101,49.0
11,SBb,143,78,45.0
15,SoA,1172,679,42.0


In [16]:
x=confronto.groupby(['true_label','train_label']).count()
pd.set_option('display.max_rows', None)
display(x)

GalaxyID
true_label train_label          
A          E0                  2
E0         E0                598
           E3                  3
           SAc                 1
           SoA                 2
           SoB                 1
E3         E0                 19
           E3                413
           SAc                 1
           SoA                 6
E6         E3                  1
           E6                 10
           S0a_eon             1
           Scd_eon             2
S0a_eon    E3                  2
           E6                  6
           S0a_eon            90
           SAb                 1
           Scd_eon             4
SAb        SAb                 6
           SAc                 2
           SoA                 2
SAc        E3                  1
           SAb                 1
           SAc                36
           SoA                 2
SBb        SBb                 2
           SoA                 1
           SoB                 1
SBc        SAc                 3
           SBb                 3
           SBc                 4
Scd_eon    E6                  1
           S0a_eon             2
           SBc                 1
           SBd                 1
           Scd_eon            34
SoA        E0                 20
           E3                 16
           SAc                 1
           SoA                41
SoB        E0                  1
           E3                  3
           SBb                 1
           SoB                17